In [ ]:
pip install pyspark


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql.functions import sum, avg

#sc = SparkContext("local", "smoking_drinking_dataset_Ver01")

spark = SparkSession.builder.appName("Smokers_Drinkers").getOrCreate()

# Where the CSV document is

path = "/content/smoking_drinking_dataset_Ver01.csv"

df = spark.read.csv(path, header = True, inferSchema = True)

df.createOrReplaceTempView("S_D_Data")

#DRINKING and SMOKING tendencies

#drinker females
drinkers_data_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_females
FROM S_D_Data
WHERE sex = 'Female' AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#drinkers_data_female.show()

#smoker females
smokers_data_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_females
FROM S_D_Data
WHERE sex = 'Female' AND SMK_stat_type_cd = 3
GROUP BY sex, age
ORDER BY age ASC
""")
#smokers_data_female.show()

#drinker males
drinkers_data_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_males
FROM S_D_Data
WHERE sex = 'Male' AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#drinkers_data_male.show()

#smoker males
smokers_data_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_males
FROM S_D_Data
WHERE sex = 'Male' AND SMK_stat_type_cd = 3
GROUP BY sex, age
ORDER BY age ASC
""")
#smokers_data_male.show()

# Extracting data for plotting
a = drinkers_data_female.select("age").rdd.flatMap(lambda x: x).collect()
b = drinkers_data_female.select("number_of_females").rdd.flatMap(lambda x: x).collect()
c = smokers_data_female.select("number_of_females").rdd.flatMap(lambda x: x).collect()

x = drinkers_data_male.select("age").rdd.flatMap(lambda x: x).collect()
y = drinkers_data_male.select("number_of_males").rdd.flatMap(lambda x: x).collect()
z = smokers_data_male.select("number_of_males").rdd.flatMap(lambda x: x).collect()

# Plotting the bar graph
plt.bar(a, b, color='blue')
plt.xlabel('Age')
plt.ylabel('Number of Females')
plt.title('Number of Female Drinkers by Age')
plt.show()

plt.bar(a, c, color='green')
plt.xlabel('Age')
plt.ylabel('Number of Females')
plt.title('Number of Female Smokers by Age')
plt.show()

plt.bar(x, y, color='red')
plt.xlabel('Age')
plt.ylabel('Number of Males')
plt.title('Number of male Drinkers by Age')
plt.show()

plt.bar(x, z, color='violet')
plt.xlabel('Age')
plt.ylabel('Number of Males')
plt.title('Number of male Smokers by Age')
plt.show()

# Data about the blood contents is crucial to compare to healthy levels

#BLOOD PRESSURE

blood_pressure_data = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`SBP`), 2) AS Systolic_blood_pressure_mmHg,
       ROUND(AVG(`DBP`), 2) AS Diastolic_blood_pressure_mmHg
FROM S_D_Data
GROUP BY sex, age
ORDER BY sex, age ASC
""")
# blood_pressure_data.show(30)

# Blood pressure for drinkers and smokers female

blood_pressure_data_s_and_d_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`SBP`), 2) AS Systolic_blood_pressure_mmHg,
       ROUND(AVG(`DBP`), 2) AS Diastolic_blood_pressure_mmHg
FROM S_D_Data
WHERE sex = 'Female' AND SMK_stat_type_cd = 3 AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#blood_pressure_data_s_and_d_female.show(15)

# Blood pressure for non_drinkers and non_smokers female

blood_pressure_data_ns_and_nd_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`SBP`), 2) AS Systolic_blood_pressure_mmHg,
       ROUND(AVG(`DBP`), 2) AS Diastolic_blood_pressure_mmHg
FROM S_D_Data
WHERE sex = 'Female' AND SMK_stat_type_cd = 1 AND DRK_YN = 'N'
GROUP BY sex, age
ORDER BY age ASC
""")
#blood_pressure_data_ns_and_nd_female.show(15)

# Blood pressure for drinkers and smokers male

blood_pressure_data_s_and_d_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`SBP`), 2) AS Systolic_blood_pressure_mmHg,
       ROUND(AVG(`DBP`), 2) AS Diastolic_blood_pressure_mmHg
FROM S_D_Data
WHERE sex = 'Male' AND SMK_stat_type_cd = 3 AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#blood_pressure_data_s_and_d_male.show(15)

# Blood pressure for non-drinkers and non-smokers male

blood_pressure_data_ns_and_nd_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`SBP`), 2) AS Systolic_blood_pressure_mmHg,
       ROUND(AVG(`DBP`), 2) AS Diastolic_blood_pressure_mmHg
FROM S_D_Data
WHERE sex = 'Male' AND SMK_stat_type_cd = 1 AND DRK_YN = 'N'
GROUP BY sex, age
ORDER BY age ASC
""")
#blood_pressure_data_ns_and_nd_male.show(15)

#MATPLOTLIB
'''
# for males (assigning the coordinate system axis values)

x = blood_pressure_data_s_and_d_male.select('age').toPandas()['age'].to_numpy()
y = blood_pressure_data_s_and_d_male.select('Systolic_blood_pressure_mmHg').toPandas()['Systolic_blood_pressure_mmHg'].to_numpy()
z = blood_pressure_data_s_and_d_male.select('Diastolic_blood_pressure_mmHg').toPandas()['Diastolic_blood_pressure_mmHg'].to_numpy()
t = blood_pressure_data_ns_and_nd_male.select('Systolic_blood_pressure_mmHg').toPandas()['Systolic_blood_pressure_mmHg'].to_numpy()
u = blood_pressure_data_ns_and_nd_male.select('Diastolic_blood_pressure_mmHg').toPandas()['Diastolic_blood_pressure_mmHg'].to_numpy()

# for females

a = blood_pressure_data_s_and_d_female.select('age').toPandas()['age'].to_numpy()
b = blood_pressure_data_s_and_d_female.select('Systolic_blood_pressure_mmHg').toPandas()['Systolic_blood_pressure_mmHg'].to_numpy()
c = blood_pressure_data_s_and_d_female.select('Diastolic_blood_pressure_mmHg').toPandas()['Diastolic_blood_pressure_mmHg'].to_numpy()
d = blood_pressure_data_ns_and_nd_female.select('Systolic_blood_pressure_mmHg').toPandas()['Systolic_blood_pressure_mmHg'].to_numpy()
e = blood_pressure_data_ns_and_nd_female.select('Diastolic_blood_pressure_mmHg').toPandas()['Diastolic_blood_pressure_mmHg'].to_numpy()

# Plotting

plt.figure(figsize=(10, 6))

# Plot for Systolic Blood Pressure

plt.plot(x, y, label='Male - SBP(s_and_d)', marker='o')
plt.plot(a, b, label='Female - SBP(s_and_d)', marker='o')
plt.plot(x, t, label='Male - SBP(ns_and_nd)', marker='o')
plt.plot(a, d, label='Female - SBP(ns_and_nd)', marker='o')

# Customize the plot

plt.title('SBP for s_and_d / ns_and_nd')
plt.xlabel('Age')
plt.ylabel('Pressure (mmHg)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for Diastolic Blood Pressure

plt.plot(x, z, label='Male - DBP(s_and_d)', marker='o')
plt.plot(a, c, label='Female - DBP(s_and_d)', marker='o')
plt.plot(x, u, label='Male - DBP(ns_and_nd)', marker='o')
plt.plot(a, e, label='Female - DBP(ns_and_nd)', marker='o')

# Customize the plot

plt.title('DBP for s_and_d / ns_and_nd')
plt.xlabel('Age')
plt.ylabel('Pressure (mmHg)')
plt.legend()
plt.grid(True)
plt.show()
'''

#EYESIGHT AND DRINKING

# eyesight for drinkers and smokers male

eyesight_data_s_and_d_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`sight_left`), 2) AS Snellen_value_left,
       ROUND(AVG(`sight_right`), 2) AS Snellen_value_right
FROM S_D_Data
WHERE sex = 'Female' AND SMK_stat_type_cd = 3 AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#eyesight_data_s_and_d_female.show(15)

# eyesight for non_drinkers and non_smokers female

eyesight_data_ns_and_nd_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`sight_left`), 2) AS Snellen_value_left,
       ROUND(AVG(`sight_right`), 2) AS Snellen_value_right
FROM S_D_Data
WHERE sex = 'Female' AND SMK_stat_type_cd = 1 AND DRK_YN = 'N'
GROUP BY sex, age
ORDER BY age ASC
""")
#eyesight_data_ns_and_nd_female.show(15)

# eyesight for drinkers and smokers male

eyesight_data_s_and_d_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`sight_left`), 2) AS Snellen_value_left,
       ROUND(AVG(`sight_right`), 2) AS Snellen_value_right
FROM S_D_Data
WHERE sex = 'Male' AND SMK_stat_type_cd = 3 AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#eyesight_data_s_and_d_male.show(15)

# eyesight for non-drinkers and non-smokers male

eyesight_data_ns_and_nd_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`sight_left`), 2) AS Snellen_value_left,
       ROUND(AVG(`sight_right`), 2) AS Snellen_value_right
FROM S_D_Data
WHERE sex = 'Male' AND SMK_stat_type_cd = 1 AND DRK_YN = 'N'
GROUP BY sex, age
ORDER BY age ASC
""")
#eyesight_data_ns_and_nd_male.show(15)

'''
#MATPLOTLIB

# for males

x = eyesight_data_s_and_d_male.select('age').toPandas()['age'].to_numpy()
y = eyesight_data_s_and_d_male.select('Snellen_value_left').toPandas()['Snellen_value_left'].to_numpy()
z = eyesight_data_s_and_d_male.select('Snellen_value_right').toPandas()['Snellen_value_right'].to_numpy()
t = eyesight_data_ns_and_nd_male.select('Snellen_value_left').toPandas()['Snellen_value_left'].to_numpy()
u = eyesight_data_ns_and_nd_male.select('Snellen_value_right').toPandas()['Snellen_value_right'].to_numpy()

# for females

a = eyesight_data_s_and_d_female.select('age').toPandas()['age'].to_numpy()
b = eyesight_data_s_and_d_female.select('Snellen_value_left').toPandas()['Snellen_value_left'].to_numpy()
c = eyesight_data_s_and_d_female.select('Snellen_value_right').toPandas()['Snellen_value_right'].to_numpy()
d = eyesight_data_ns_and_nd_female.select('Snellen_value_left').toPandas()['Snellen_value_left'].to_numpy()
e = eyesight_data_ns_and_nd_female.select('Snellen_value_right').toPandas()['Snellen_value_right'].to_numpy()

# Plotting

plt.figure(figsize=(10, 6))

# Plot for eyesight

plt.plot(x, y, label='Male - eyesight_left(s_and_d)', marker='o')
plt.plot(a, b, label='Female - eyesight_left(s_and_d)', marker='o')
plt.plot(x, t, label='Male - eyesight_left(ns_and_nd)', marker='o')
plt.plot(a, d, label='Female - eyesight_left(ns_and_nd)', marker='o')

# Customize the plot

plt.title('Eyesight_left')
plt.xlabel('Age')
plt.ylabel('Snellen value')
plt.legend()
plt.grid(True)
plt.show()

# Plot for Diastolic Blood Pressure

plt.plot(x, z, label='Male - eyesight_right(s_and_d)', marker='o')
plt.plot(a, c, label='Female - eyesight_right(s_and_d)', marker='o')
plt.plot(x, u, label='Male - eyesight_right(ns_and_nd)', marker='o')
plt.plot(a, e, label='Female - eyesight_right(ns_and_nd)', marker='o')

# Customize the plot

plt.title('Eyesight_right')
plt.xlabel('Age')
plt.ylabel('Snellen value')
plt.legend()
plt.grid(True)
plt.show()
'''

# WEIGHT

weight_data_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(weight), 2) AS average_weight
FROM S_D_Data
WHERE sex = 'Female'
GROUP BY sex, age
ORDER BY age ASC
""")
#weight_data_female.show(30)

weight_data_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(weight), 2) AS average_weight
FROM S_D_Data
WHERE sex = 'Male'
GROUP BY sex, age
ORDER BY age ASC
""")
#weight_data_male.show(30)

#MATPLOTLIB
'''
# for males

x = weight_data_male.select('age').toPandas()['age'].to_numpy()
y = weight_data_male.select('average_weight').toPandas()['average_weight'].to_numpy()

# for females

a = weight_data_female.select('age').toPandas()['age'].to_numpy()
b = weight_data_female .select('average_weight').toPandas()['average_weight'].to_numpy()

# Plotting

plt.figure(figsize=(10, 6))

# Plot for weight

plt.plot(x, y, label='Male - average weight', marker='o')
plt.plot(a, b, label='Female - average weight', marker='o')

# Customize the plot

plt.title('Average weight by Age and Sex')
plt.xlabel('Age')
plt.ylabel('Weight(kg)')
plt.legend()
plt.grid(True)
plt.show()
'''

cholesterol_data = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`BLDS`), 2) AS BLDS_fasting_blood_glucose_mgdL,
       ROUND(AVG(`tot_chole`), 2) AS total_cholesterol_mgdL,
       ROUND(AVG(`HDL_chole`), 2) AS HDL_cholesterol_mgdL
FROM S_D_Data
GROUP BY sex, age
ORDER BY sex, age ASC
""")
# cholesterol_data.show(30)

cholesterol_data_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`BLDS`), 2) AS BLDS_fasting_blood_glucose_mgdL,
       ROUND(AVG(`tot_chole`), 2) AS total_cholesterol_mgdL,
       ROUND(AVG(`HDL_chole`), 2) AS HDL_cholesterol_mgdL,
       ROUND(AVG(`LDL_chole`), 2) AS LDL_cholesterol_mgdL,
        ROUND(AVG(`triglyceride`), 2) AS triglyceride_mgdL,
       ROUND(AVG(`serum_creatinine`), 2) AS serum_creatinine_mgdL
FROM S_D_Data
WHERE sex = 'Female'
GROUP BY sex, age
ORDER BY age ASC
""")
#cholesterol_data_female.show(15)

cholesterol_data_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`BLDS`), 2) AS BLDS_fasting_blood_glucose_mgdL,
       ROUND(AVG(`tot_chole`), 2) AS total_cholesterol_mgdL,
       ROUND(AVG(`HDL_chole`), 2) AS HDL_cholesterol_mgdL,
       ROUND(AVG(`LDL_chole`), 2) AS LDL_cholesterol_mgdL,
       ROUND(AVG(`triglyceride`), 2) AS triglyceride_mgdL,
       ROUND(AVG(`serum_creatinine`), 2) AS serum_creatinine_mgdL
FROM S_D_Data
WHERE sex = 'Male'
GROUP BY sex, age
ORDER BY age ASC
""")
#cholesterol_data_male.show(15)

#MATPLOTLIB
'''
# for males

x = cholesterol_data_male.select('age').toPandas()['age'].to_numpy()
y = cholesterol_data_male.select('BLDS_fasting_blood_glucose_mgdL').toPandas()['BLDS_fasting_blood_glucose_mgdL'].to_numpy()
z = cholesterol_data_male.select('total_cholesterol_mgdL').toPandas()['total_cholesterol_mgdL'].to_numpy()
m = cholesterol_data_male.select('HDL_cholesterol_mgdL').toPandas()['HDL_cholesterol_mgdL'].to_numpy()
n = cholesterol_data_male.select('LDL_cholesterol_mgdL').toPandas()['LDL_cholesterol_mgdL'].to_numpy()
o = cholesterol_data_male.select('triglyceride_mgdL').toPandas()['triglyceride_mgdL'].to_numpy()
p = cholesterol_data_male.select('serum_creatinine_mgdL').toPandas()['serum_creatinine_mgdL'].to_numpy()

# Extracting data for females

a = cholesterol_data_female.select('age').toPandas()['age'].to_numpy()
b = cholesterol_data_female.select('BLDS_fasting_blood_glucose_mgdL').toPandas()['BLDS_fasting_blood_glucose_mgdL'].to_numpy()
c = cholesterol_data_female.select('total_cholesterol_mgdL').toPandas()['total_cholesterol_mgdL'].to_numpy()
d = cholesterol_data_female.select('HDL_cholesterol_mgdL').toPandas()['HDL_cholesterol_mgdL'].to_numpy()
e = cholesterol_data_female.select('LDL_cholesterol_mgdL').toPandas()['LDL_cholesterol_mgdL'].to_numpy()
f = cholesterol_data_female.select('triglyceride_mgdL').toPandas()['triglyceride_mgdL'].to_numpy()
g = cholesterol_data_female.select('serum_creatinine_mgdL').toPandas()['serum_creatinine_mgdL'].to_numpy()

# Plotting

plt.figure(figsize=(10, 6))

# Plot for BLDS

plt.plot(x, y, label='Male - BLDS', marker='o')
plt.plot(a, b, label='Female - BLDS', marker='o')

plt.title('Average BLDS by Age and Sex')
plt.xlabel('Age')
plt.ylabel('Amount(mg/dL)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for total_cholesterol

plt.plot(x, z, label='Male - total_cholesterol', marker='o')
plt.plot(a, c, label='Female - total_cholesterol', marker='o')

plt.title('Average total_cholesterol by Age and Sex')
plt.xlabel('Age')
plt.ylabel('Amount(mg/dL)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for HDL

plt.plot(x, m, label='Male - HDL', marker='o')
plt.plot(a, d, label='Female - HDL', marker='o')

plt.title('Average HDL by Age and Sex')
plt.xlabel('Age')
plt.ylabel('Amount(mg/dL)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for LDL

plt.plot(x, n, label='Male - LDL', marker='o')
plt.plot(a, e, label='Female - LDL', marker='o')

plt.title('Average LDL by Age and Sex')
plt.xlabel('Age')
plt.ylabel('Amount(mg/dL)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for triglyceride

plt.plot(x, o, label='Male - triglyceride', marker='o')
plt.plot(a, f, label='Female - triglyceride', marker='o')

plt.title('Average triglyceride by Age and Sex')
plt.xlabel('Age')
plt.ylabel('Amount(mg/dL)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for serum_creatinine

plt.plot(x, p, label='Male - serum_creatinine', marker='o')
plt.plot(a, g, label='Female - serum_creatinine', marker='o')

plt.title('Average serum_creatinine by Age and Sex')
plt.xlabel('Age')
plt.ylabel('Amount(mg/dL)')
plt.legend()
plt.grid(True)
plt.show()
'''

#BLDS

#BLDS for smokers and drinkers female

BLDS_data_s_and_d_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`BLDS`), 2) AS BLDS_s_and_d_mgdL
FROM S_D_Data
WHERE sex = 'Female' AND SMK_stat_type_cd = 3 AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#BLDS_data_s_and_d_female.show(15)

#BLDS for non-smokers and drinkers female

BLDS_data_ns_and_nd_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`BLDS`), 2) AS BLDS_ns_and_nd_mgdL
FROM S_D_Data
WHERE sex = 'Female' AND SMK_stat_type_cd = 1 AND DRK_YN = 'N'
GROUP BY sex, age
ORDER BY age ASC
""")
#BLDS_data_ns_and_nd_female.show(15)

#BLDS for smokers and drinkers male

BLDS_data_s_and_d_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`BLDS`), 2) AS BLDS_s_and_d_mgdL
FROM S_D_Data
WHERE sex = 'Male' AND SMK_stat_type_cd = 3 AND DRK_YN = 'Y'
GROUP BY sex, age
ORDER BY age ASC
""")
#BLDS_data_s_and_d_male.show(15)

#BLDS for non-smokers and drinkers male

BLDS_data_ns_and_nd_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`BLDS`), 2) AS BLDS_ns_and_nd_mgdL
FROM S_D_Data
WHERE sex = 'Male' AND SMK_stat_type_cd = 1 AND DRK_YN = 'N'
GROUP BY sex, age
ORDER BY age ASC
""")
#BLDS_data_ns_and_nd_male.show(15)
'''
#MATPLOTLIB

# for males

x = BLDS_data_s_and_d_male.select('age').toPandas()['age'].to_numpy()
y = BLDS_data_s_and_d_male.select('BLDS_s_and_d_mgdL').toPandas()['BLDS_s_and_d_mgdL'].to_numpy()
z = BLDS_data_ns_and_nd_male.select('BLDS_ns_and_nd_mgdL').toPandas()['BLDS_ns_and_nd_mgdL'].to_numpy()
# for females

a = BLDS_data_s_and_d_female.select('age').toPandas()['age'].to_numpy()
b = BLDS_data_s_and_d_female.select('BLDS_s_and_d_mgdL').toPandas()['BLDS_s_and_d_mgdL'].to_numpy()
c = BLDS_data_ns_and_nd_female.select('BLDS_ns_and_nd_mgdL').toPandas()['BLDS_ns_and_nd_mgdL'].to_numpy()

# Plotting

plt.figure(figsize=(10, 6))

# Plot for BLDS (drinkers and smokers)

plt.plot(x, y, label='Male - BLDS (d and s)', marker='o')
plt.plot(a, b, label='Female - BLDS (d and s)', marker='o')

# Plot for BLDS (non-drinkers and non-smokers)

plt.plot(x, z, label='Male - BLDS (nd and ns)', marker='o')
plt.plot(a, c, label='Female - BLDS (nd and ns)', marker='o')

plt.title('Average BLDS')
plt.xlabel('Age')
plt.ylabel('Amount(mg/dL)')
plt.legend()
plt.grid(True)
plt.show()
'''

hemoglobin_data_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`hemoglobin`), 2) AS hemoglobin_gdL
FROM S_D_Data
WHERE sex = 'Female'
GROUP BY sex, age
ORDER BY age ASC
""")
#hemoglobin_data_female.show(15)

hemoglobin_data_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people,
       ROUND(AVG(`hemoglobin`), 2) AS hemoglobin_gdL
FROM S_D_Data
WHERE sex = 'Male'
GROUP BY sex, age
ORDER BY age ASC
""")
#hemoglobin_data_male.show(15)

smoker_or_not_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people, SMK_stat_type_cd
FROM S_D_Data
WHERE sex = 'Female'
GROUP BY sex, age, SMK_stat_type_cd
ORDER BY age ASC
""")
# smoker_or_not_female.show(50)

smoker_or_not_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people, SMK_stat_type_cd
FROM S_D_Data
WHERE sex = 'Male'
GROUP BY sex, age, SMK_stat_type_cd
ORDER BY age ASC
""")
# smoker_or_not_male.show(50)

drinker_or_not_female = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people, DRK_YN
FROM S_D_Data
WHERE sex = 'Female'
GROUP BY sex, age, DRK_YN
ORDER BY age ASC
""")
#drinker_or_not_female.show(50)

drinker_or_not_male = spark.sql("""
SELECT sex, age, COUNT(*) as number_of_people, DRK_YN
FROM S_D_Data
WHERE sex = 'Male'
GROUP BY sex, age, DRK_YN
ORDER BY age ASC
""")
#drinker_or_not_male.show(50)


In [38]:
spark.stop()